In [12]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.lines import Line2D
from matplotlib.patches import Rectangle
from IPython.display import HTML
import matplotlib.patheffects as pe

# Set visual style for scientific validation
sns.set_theme(style="whitegrid", context="talk")
plt.rcParams['figure.figsize'] = (12, 6)

In [27]:
# Load Master Data
INPUT_FILE = '../data/processed/void_metrics_by_play.csv'
void_results = pd.read_csv(INPUT_FILE, low_memory=False)

In [28]:
df.columns

Index(['game_id', 'play_id', 'nfl_id', 'player_name', 'player_position',
       'player_role', 'zone_assignment'],
      dtype='object')

In [30]:
import pandas as pd
import numpy as np

# ------------------------------------------------------------------------------
# 1. LOAD DATA
# ------------------------------------------------------------------------------
# Load the Metrics (The "Scorecard")
df_metrics = pd.read_csv('../data/processed/void_metrics_by_play.csv')

# Load the Context (To get Coverage Type)
# We only load necessary columns to save memory
df_context = pd.read_csv('../data/processed/tracking_clean_normalized.csv', 
                         usecols=['game_id', 'play_id', 'team_coverage_type'])

# Deduplicate context (since tracking data has multiple frames per play)
df_context = df_context.drop_duplicates(subset=['game_id', 'play_id'])

# Merge Coverage Type into the Metrics
df_merged = df_metrics.merge(df_context, on=['game_id', 'play_id'], how='left')

# Create a unique play identifier for accurate counting
df_merged['unique_play_id'] = df_merged['game_id'].astype(str) + '_' + df_merged['play_id'].astype(str)

# ------------------------------------------------------------------------------
# 2. GENERATE TABLE A: THE COST OF DISCIPLINE (Player Rankings)
# ------------------------------------------------------------------------------
# Group by Player and Position
table_a = df_merged.groupby(['player_name', 'player_position']).agg(
    total_snaps=('unique_play_id', 'count'),
    void_penalties=('void_penalty', 'sum'),
    total_epa_lost=('damage_epa', 'sum'),
    avg_drift_yards=('drift_yards', 'mean')
).reset_index()

# Calculate Void Rate %
table_a['void_rate_pct'] = (table_a['void_penalties'] / table_a['total_snaps']) * 100

# Sort by Total EPA Lost (Descending) - showing the most "expensive" players first
table_a = table_a.sort_values('total_epa_lost', ascending=False)

# Filter: Remove noise (e.g., players with fewer than 10 snaps)
table_a_filtered = table_a[table_a['total_snaps'] >= 10].reset_index(drop=True)

# Formatting for Display
print("\n=== TABLE A: THE COST OF DISCIPLINE (Top 15 Expensive Players) ===")
display_cols_a = ['player_name', 'player_position', 'total_snaps', 
                  'void_penalties', 'void_rate_pct', 'total_epa_lost', 'avg_drift_yards']
print(table_a_filtered[display_cols_a].head(15).to_markdown(index=False, floatfmt=".2f"))


# ------------------------------------------------------------------------------
# 3. GENERATE TABLE D: ZONE INTEGRITY BY COVERAGE SHELL
# ------------------------------------------------------------------------------
# Group by Scheme (Coverage Type)
table_d = df_merged.groupby('team_coverage_type').agg(
    unique_plays=('unique_play_id', 'nunique'),
    player_snaps=('unique_play_id', 'count'),
    total_voids=('void_penalty', 'sum'),
    total_epa_lost=('damage_epa', 'sum')
).reset_index()

# Calculate Metrics
table_d['void_freq_pct'] = (table_d['total_voids'] / table_d['player_snaps']) * 100
table_d['epa_lost_per_void'] = table_d['total_epa_lost'] / table_d['total_voids']

# Sort by Void Frequency (How often does this scheme break?)
table_d = table_d.sort_values('void_freq_pct', ascending=False)

# Formatting for Display
print("\n=== TABLE D: ZONE INTEGRITY BY COVERAGE SHELL ===")
print(table_d.to_markdown(index=False, floatfmt=".4f"))

# ------------------------------------------------------------------------------
# 4. EXPORT (Optional)
# ------------------------------------------------------------------------------
# table_a_filtered.to_csv('data/processed/report_table_A_player_rankings.csv', index=False)
# table_d.to_csv('data/processed/report_table_D_coverage_integrity.csv', index=False)


=== TABLE A: THE COST OF DISCIPLINE (Top 15 Expensive Players) ===
| player_name         | player_position   |   total_snaps |   void_penalties |   void_rate_pct |   total_epa_lost |   avg_drift_yards |
|:--------------------|:------------------|--------------:|-----------------:|----------------:|-----------------:|------------------:|
| Zaire Franklin      | MLB               |            54 |               12 |           22.22 |            40.06 |              4.57 |
| Kenny Moore II      | CB                |            32 |               24 |           75.00 |            38.01 |              8.33 |
| C.J. Mosley         | ILB               |            36 |               10 |           27.78 |            38.00 |              5.39 |
| Logan Wilson        | MLB               |            45 |               12 |           26.67 |            37.93 |              4.33 |
| Robert Spillane     | ILB               |            43 |               11 |           25.58 |            36.65 | 

In [43]:
import pandas as pd
import numpy as np

# 1. Load Data
df_metrics = pd.read_csv('../data/processed/void_metrics_by_play.csv')

# ==============================================================================
# TABLE B: THE "BAIL OUT" INDEX (Who gets saved by teammates?)
# ==============================================================================
# Logic: We identify "Risk Events" where a player abandoned their zone (Drifted > 5 & Dist > 3).
# We then check how often they were "Bailed Out" (Help < 4 yds) vs "Penalized" (No Help).
# High "Safety Net %" means the player relies heavily on teammates to cover their mistakes.

df_metrics['risk_event'] = df_metrics['void_penalty'] | df_metrics['bailed_out']

table_b = df_metrics[df_metrics['risk_event']].groupby(['player_name', 'player_position']).agg(
    total_risk_events=('risk_event', 'sum'),
    bailed_out_count=('bailed_out', 'sum'),
    void_penalty_count=('void_penalty', 'sum'),
    avg_help_dist=('closest_help_dist', 'mean')
).reset_index()

table_b['safety_net_pct'] = (table_b['bailed_out_count'] / table_b['total_risk_events']) * 100

# Filter for volume (at least 5 risk events) and sort by "Most Saved"
table_b = table_b.sort_values('total_risk_events', ascending=False)
table_b_filtered = table_b[table_b['total_risk_events'] >= 5].reset_index(drop=True)
table_b
# print("\n=== TABLE B: THE BAIL OUT INDEX (Top 10 High-Risk Players) ===")
# print(table_b_filtered.head(10).to_markdown(index=False, floatfmt=".2f"))

# # ==============================================================================
# # TABLE C: POSITIONAL HAZARD REPORT (Linebacker Conflict)
# # ==============================================================================
# # Logic: Quantify the difficulty of playing Linebacker in Zone vs DBs.
# # We expect LBs to have higher void rates due to conflict (Run/Pass reads).

# table_c = df_metrics.groupby('player_position').agg(
#     total_snaps=('game_id', 'count'),
#     total_voids=('void_penalty', 'sum'),
#     avg_drift=('drift_yards', 'mean'),
#     avg_epa_lost=('damage_epa', 'mean')
# ).reset_index()

# table_c['void_rate_pct'] = (table_c['total_voids'] / table_c['total_snaps']) * 100
# table_c = table_c.sort_values('void_rate_pct', ascending=False)

# print("\n=== TABLE C: POSITIONAL HAZARD REPORT ===")
# print(table_c.to_markdown(index=False, floatfmt=".2f"))

# # ==============================================================================
# # TABLE E: THE PRICE OF ERROR (Impact Analysis)
# # ==============================================================================
# # Logic: Compare the average EPA of plays where a Void occurred vs. Zone Intact.
# # This proves the "So What?" of the project.

# # Aggregate to Play Level (Did *anyone* have a void on this play?)
# play_level = df_metrics.groupby(['game_id', 'play_id']).agg(
#     has_void=('void_penalty', 'max'), # True if any player on defense had a void
#     epa=('epa_lost', 'max')           # EPA is constant for the play
# ).reset_index()

# table_e = play_level.groupby('has_void').agg(
#     play_count=('game_id', 'count'),
#     avg_epa=('epa', 'mean'),
#     median_epa=('epa', 'median')
# ).reset_index()

# table_e['condition'] = np.where(table_e['has_void'], 'Void Occurred', 'Zone Intact')
# table_e = table_e[['condition', 'play_count', 'avg_epa', 'median_epa']]

# print("\n=== TABLE E: THE PRICE OF ERROR (EPA Impact) ===")
# print(table_e.to_markdown(index=False, floatfmt=".4f"))

# Export
# table_b_filtered.to_csv('data/processed/report_table_B_bailout.csv', index=False)
# table_c.to_csv('data/processed/report_table_C_positions.csv', index=False)
# table_e.to_csv('data/processed/report_table_E_impact.csv', index=False)

,player_name,player_position,total_risk_events,bailed_out_count,void_penalty_count,avg_help_dist,safety_net_pct
377,T.J. Edwards,OLB,27,0,27,6.451178,0.000000
265,Kenny Moore II,CB,24,0,24,NaN,0.000000
160,Foye Oluokun,OLB,22,1,21,3.362434,4.545455
40,Bobby Wagner,MLB,19,1,18,3.598155,5.263158
264,"Kenneth Murray, Jr.",MLB,17,0,17,NaN,0.000000
...,...,...,...,...,...,...,...
114,Deane Leonard,CB,1,0,1,NaN,0.000000
411,Xavier Woods,FS,1,0,1,NaN,0.000000
22,Anthony Barr,OLB,1,0,1,NaN,0.000000
374,Starling Thomas V,CB,1,0,1,NaN,0.000000


In [44]:
import pandas as pd
import os

print("🕵️‍♀️ INVESTIGATING 'NaN HELP' (SOLITARY CONFINEMENT) BUG...")

# --- CONFIGURATION ---
# Adjust these paths if your notebook is in a different folder
BASE_DIR = '../data/processed' 
METRICS_FILE = 'void_metrics_by_play.csv'
TRACKING_FILE = 'tracking_enriched_frames.csv' # or 'master_zone_tracking.csv'

# 1. LOAD THE "CRIME REPORT"
path_metrics = os.path.join(BASE_DIR, METRICS_FILE)
if not os.path.exists(path_metrics):
    raise FileNotFoundError(f"Missing: {path_metrics}")

print(f"   Loading {METRICS_FILE}...")
void_df = pd.read_csv(path_metrics)

# 2. IDENTIFY THE SUSPECTS
# Find plays where a player was penalized but 'closest_help_dist' is NaN
nan_help_df = void_df[
    (void_df['void_penalty'] == True) & 
    (void_df['closest_help_dist'].isna())
]

if nan_help_df.empty:
    print("✅ No NaN Help Distance found! The issue is resolved.")
else:
    count = len(nan_help_df)
    print(f"⚠️ FOUND {count} PLAYS WITH 'GHOST TEAMMATES' (NaN Help Distance).")
    
    # Pick the most frequent victim (e.g., Kenny Moore II)
    suspect_name = nan_help_df['player_name'].value_counts().idxmax()
    print(f"\n🔬 Deep Dive into Suspect: {suspect_name}")
    
    # Get 3 sample plays for this suspect
    sample_plays = nan_help_df[nan_help_df['player_name'] == suspect_name].head(3)
    
    # 3. LOAD THE "SURVEILLANCE TAPE" (Raw Tracking)
    # We only load this NOW, and only looking for specific columns to save RAM
    print(f"   Loading {TRACKING_FILE} (Metadata Only)...")
    path_tracking = os.path.join(BASE_DIR, TRACKING_FILE)
    
    # We need to check if teammates exist in the raw data
    cols = ['game_id', 'play_id', 'nfl_id', 'frame_id', 'player_name', 'player_role', 'team_coverage_type']
    tracking_df = pd.read_csv(path_tracking, usecols=cols)

    print("\n--- FORENSIC ANALYSIS OF SAMPLES ---")
    for _, row in sample_plays.iterrows():
        gid, pid = row['game_id'], row['play_id']
        
        print(f"\n▶️ Game {gid} | Play {pid}")
        
        # Filter raw tracking for this play
        play_data = tracking_df[
            (tracking_df['game_id'] == gid) & 
            (tracking_df['play_id'] == pid)
        ]
        
        if play_data.empty:
            print("   ❌ CRITICAL: This play is MISSING from the Raw Tracking file!")
            continue

        # Check Coverage Type
        cov_type = play_data['team_coverage_type'].iloc[0] if 'team_coverage_type' in play_data.columns else "Unknown"
        print(f"   Coverage Type: {cov_type}")
        
        # Check the Max Frame (The moment the void calculation happens)
        max_frame = play_data['frame_id'].max()
        final_frame_data = play_data[play_data['frame_id'] == max_frame]
        
        # Count Defenders present in that final frame
        defenders = final_frame_data[final_frame_data['player_role'] == 'Defensive Coverage']
        defender_count = len(defenders)
        
        print(f"   Max Frame: {max_frame}")
        print(f"   Defenders visible at Max Frame: {defender_count}")
        
        if defender_count <= 1:
            print("   🔴 DIAGNOSIS: RAW DATA ISSUE. Teammates are missing from the source file.")
        else:
            print("   🔵 DIAGNOSIS: MERGE ISSUE. Teammates exist in raw data, but Void Script lost them.")
            print(f"   Teammates found: {defenders['player_name'].tolist()}")

🕵️‍♀️ INVESTIGATING 'NaN HELP' (SOLITARY CONFINEMENT) BUG...
   Loading void_metrics_by_play.csv...
⚠️ FOUND 1663 PLAYS WITH 'GHOST TEAMMATES' (NaN Help Distance).

🔬 Deep Dive into Suspect: T.J. Edwards
   Loading tracking_enriched_frames.csv (Metadata Only)...

--- FORENSIC ANALYSIS OF SAMPLES ---

▶️ Game 2023091008 | Play 840
   Coverage Type: COVER_2_ZONE
   Max Frame: 32
   Defenders visible at Max Frame: 2
   🔵 DIAGNOSIS: MERGE ISSUE. Teammates exist in raw data, but Void Script lost them.
   Teammates found: ['T.J. Edwards', 'Tyrique Stevenson']

▶️ Game 2023100500 | Play 3199
   Coverage Type: COVER_3_ZONE
   Max Frame: 50
   Defenders visible at Max Frame: 2
   🔵 DIAGNOSIS: MERGE ISSUE. Teammates exist in raw data, but Void Script lost them.
   Teammates found: ['T.J. Edwards', 'Tyrique Stevenson']

▶️ Game 2023100500 | Play 3224
   Coverage Type: COVER_3_ZONE
   Max Frame: 37
   Defenders visible at Max Frame: 2
   🔵 DIAGNOSIS: MERGE ISSUE. Teammates exist in raw data, but V

In [49]:
import pandas as pd
import glob
import os

print("🕵️‍♀️ CENSUS CHECK: Are all players in the output files?")

# 1. FIND A RAW OUTPUT FILE
# We grab the first available output file to test
output_files = sorted(glob.glob('../data/train/output_*.csv'))

if not output_files:
    raise FileNotFoundError("No output files found in data/train/")

target_file = output_files[2]
print(f"📂 Inspecting: {target_file}")

# 2. LOAD RAW DATA
# We only need play_id and nfl_id to count heads
df_raw = pd.read_csv(target_file, usecols=['game_id', 'play_id', 'nfl_id'])

# 3. COUNT PLAYERS PER PLAY
# We count unique nfl_id's for each play
player_counts = df_raw.groupby(['game_id', 'play_id'])['nfl_id'].nunique()

# 4. THE VERDICT
print("\n--- CENSUS RESULTS ---")
print(f"Total Plays Analyzed: {len(player_counts)}")
print(f"Median Players per Play: {player_counts.median()}")
print(f"Min Players: {player_counts.min()}")
print(f"Max Players: {player_counts.max()}")

print("\n--- SAMPLE DISTRIBUTION ---")
print(player_counts.value_counts().sort_index().head(10))

# 5. CONCLUSION GENERATOR
median_count = player_counts.median()
print(f"\n--- VERDICT ---")
if median_count > 20:
    print(f"✅ FULL DATA ({median_count} players). The bug is in your ETL/Merge logic.")
else:
    print(f"❌ SPARSE DATA ({median_count} players). The dataset does NOT contain all teammates.")

🕵️‍♀️ CENSUS CHECK: Are all players in the output files?
📂 Inspecting: ../data/train/output_2023_w03.csv

--- CENSUS RESULTS ---
Total Plays Analyzed: 904
Median Players per Play: 3.0
Min Players: 1
Max Players: 7

--- SAMPLE DISTRIBUTION ---
nfl_id
1     64
2    167
3    318
4    225
5     82
6     39
7      9
Name: count, dtype: int64

--- VERDICT ---
❌ SPARSE DATA (3.0 players). The dataset does NOT contain all teammates.
